In [2]:
!pip install mysql-connector-python

     ---------------------------------------- 7.9/7.9 MB 13.6 MB/s eta 0:00:00


In [1]:
import mysql
import mysql.connector
from typing import List, Tuple, Set, Dict
import pandas as pd
import numpy as np

In [37]:
mydb = mysql.connector.connect(
      host="localhost",
      user="jared",
      password="199800",
    )


mycursor = mydb.cursor()

# mycursor.execute("CREATE DATABASE trial_database")


mycursor.execute("SHOW DATABASES")

for each_db in mycursor:
    print(each_db)

('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('trial_database',)
('world',)


#### TODO
1. Create Database skeleton (class)
2. DB class should have creation api
3. Add podcast API
4. Modify DB API (CRUD)

In [37]:
class Podcast_DB():
    def __init__(self, db_name : str, threshold : int, credentials : Tuple[str, str] = None, create : bool = False):
        """
        credentials: tuple of (username, password)
        
        function:
        
        """
    
        username = "jared" if not credentials else credentials[0]
        pw = "199800" if not credentials else credentials[1]
        
        self.db = mysql.connector.connect(
                  host="localhost",
                  user=username,
                  password=pw
        )
        print("Established MySQL connection with host.")

        if create:
            mycursor = self.db.cursor()
            mycursor.execute("SHOW DATABASES")
            for each_db in mycursor:
                if db_name == each_db[0]:
                    print(f"You are creating a database {db_name} that already exists! Please set create = False (default).")
                    return 

            database_sql = f"CREATE DATABASE {db_name}"


        else:
            mycursor = self.db.cursor()
            mycursor.execute("SHOW DATABASES")
            exists = False
            for each_db in mycursor:
                if db_name == each_db[0]:
                    exists = True
                    break

            if not exists:
                print(f"You are trying to access a database with name: {db_name} that does not exist. Please set create = True.")
                return

        self.db = mysql.connector.connect(
              host="localhost",
              user=username,
              password=pw,
              database = db_name
            )

        print(f"Established MySQL connection with database {db_name}.")


        self.threshold = threshold
################################################# TABLE NAMES ###################################################
        self.first_table = "podcast_views"
        self.second_table = "podcast_details"
        
#         Checks whether tables are in database already 
        first_t_cnt = 0
        sec_t_cnt = 0
        
        mycursor = self.db.cursor()
        mycursor.execute("SHOW TABLES")
        for each_name in mycursor:
            if self.first_table == each_name[0]:
                first_t_cnt += 1
            elif self.second_table == each_name[0]:
                sec_t_cnt += 1

        if not first_t_cnt:
            mycursor.execute(f"CREATE TABLE {self.first_table} (pid INT NOT NULL AUTO_INCREMENT PRIMARY KEY, num_views INT)")
        if not sec_t_cnt:
            second_table_sql = f"""CREATE TABLE {self.second_table} 
            (pid INT NOT NULL AUTO_INCREMENT PRIMARY KEY, 
            url VARCHAR(255), 
            podcast_title VARCHAR(255), 
            podcast_name VARCHAR(255), 
            transcript_filepath VARCHAR(255), 
            embeddings_filepath VARCHAR(255), 
            full_summary_filepath VARCHAR(255),
            summary VARCHAR(4096))"""

            mycursor.execute(second_table_sql)
            
        print(f"Tables successfully created or are present in database {db_name}.")
        
    
    def show_db(self, columns : List[str]):
        column_names = ", ".join(columns)
        
        mycursor = self.db.cursor()
        mycursor.execute(f"select {column_names} from {self.second_table}")
        myresult = mycursor.fetchall()
        
        all_entries = []
        for each_entry in myresult:
            all_entries.append(each_entry)
            print(each_entry)
            
        return all_entries
    
    def query_db(self, query : str):
        """use query = all to select ALL
        else put in custom query
        returns: list of tuples with each tuple representing a row in SQL datatable
        """
        
        if query == "all":
            query = f"SELECT * from {self.second_table}"
        
        mycursor = self.db.cursor()
        mycursor.execute(query)
        myresult = mycursor.fetchall()

        all_entries = []
        for each_entry in myresult:
            all_entries.append(each_entry)
            print(each_entry)
            
        return all_entries
    
    
    def update_db(self, pid : int, update_dict : Dict[str, str]):
        """
        update_dict : {col_name : new_value} for given pid
        """
        
        all_updates = []
        for key, val in update_dict.items():
            each_update = key + " = " + "'" + val + "'"
            all_updates.append(each_update)
        
        update_str = ", ".join(all_updates)
        where_cond = f"pid = {pid}"
        
        update_query = f"""UPDATE {self.second_table} SET {update_str} WHERE {where_cond}"""
        
        print(update_query)
        mycursor = self.db.cursor()
        mycursor.execute(update_query)
        self.db.commit()
        
        print(f"Record with pid = {pid} has been successfully updated.") 
        
        return

    
    def insert_podcast(self, podcast_params : List[Tuple[str]], incl_pid = False, which_table = "second"):
        """
        insert List of tuples, where each tuple contains podcast_parameters
                (url VARCHAR(255), 
                podcast_title VARCHAR(255), 
                podcast_name VARCHAR(255), 
                transcript_filepath VARCHAR(255), 
                embeddings_filepath VARCHAR(255), 
                full_summary_filepath VARCHAR(255),
                summary VARCHAR(4096))
        
        incl_pid = True means pid will be included as first argument as well so one more param in tuple. Default False.
        
        No need for pid as it is primary key and auto incremented. Can still include but must make sure pid does not already exist
        in table.
        
        """
        table_name = self.first_table if which_table == "first" else self.second_table
        print(table_name)
        mycursor = self.db.cursor()
        
        if which_table == "first":
            if incl_pid:
                sql = f"""INSERT INTO {table_name} (pid, num_views) VALUES (%s, %s)"""
            else:
                sql = f"""INSERT INTO {table_name} (num_views) VALUES (%s)"""

        else:

            if incl_pid:
                sql = f"""INSERT INTO {table_name} (pid, url, podcast_title, podcast_name, transcript_filepath, embeddings_filepath,
                        full_summary_filepath, summary) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
            else:
                sql = f"""INSERT INTO {table_name} (url, podcast_title, podcast_name, transcript_filepath, embeddings_filepath,
                            full_summary_filepath, summary) VALUES (%s, %s, %s, %s, %s, %s, %s)"""

        mycursor.executemany(sql, podcast_params)

            
        self.db.commit()
        print(f"{mycursor.rowcount} records successfully inserted.")
        
        
    
    def balance_db(self) -> None:
#         Balances both dataframes by ensuring second dataframe only has podcasts above the specified threshold
#         Minimally both tables would have been initialized even if empty due to __init__ method
        relevant_ids = self.query_db(f"SELECT pid from {self.first_table} WHERE num_views >= {self.threshold}")
#     Only include relevant ids in second table to reduce storage
#         Delete all rows in second table that are not in id_set
        id_set = {each_id[0] for each_id in relevant_ids}
        
        delete_query = f"""DELETE FROM {self.second_table} WHERE pid NOT IN {id_set}"""
        
        print(delete_query)
        mycursor = self.db.cursor()
        mycursor.execute(delete_query)
        self.db.commit()
        
        print(f"{mycursor.rowcount} records have been succesfully filtered out due to threshold of {self.threshold}.")
        
                            
    


In [44]:
podcast_db = Podcast_DB(db_name = "trial_database", threshold = 100)

Established MySQL connection with host.
Established MySQL connection with database trial_database.
Tables successfully created or are present in database trial_database.


In [12]:

sample_podcast_1 = ("https://some_podcast.com", "some_podcast_title", "some_podcast_name", "dir/transcript/path", "dir/embedding/path",
                 "dir/summary_path", "This is a great summary!")

sample_podcast_2 = ("https://other_podcast.com", "other_podcast_title", "other_podcast_name", "dir/transcript/otherpath", 
                    "dir/embedding/otherpath", "dir/summary_otherpath", "This is a great other summary!")

podcast_lst = [sample_podcast_1, sample_podcast_2]

# podcast_db.insert_podcast(podcast_lst)

In [45]:
podcast_db.query_db("all")

(1, 'https://newurl.com', 'some_podcast_title', 'nicer_name', 'dir/transcript/path', 'dir/embedding/path', 'dir/summary_path', 'A much better summary today!')
(2, 'https://other_podcast.com', 'other_podcast_title', 'other_podcast_name', 'dir/transcript/otherpath', 'dir/embedding/otherpath', 'dir/summary_otherpath', 'This is a great other summary!')


[(1,
  'https://newurl.com',
  'some_podcast_title',
  'nicer_name',
  'dir/transcript/path',
  'dir/embedding/path',
  'dir/summary_path',
  'A much better summary today!'),
 (2,
  'https://other_podcast.com',
  'other_podcast_title',
  'other_podcast_name',
  'dir/transcript/otherpath',
  'dir/embedding/otherpath',
  'dir/summary_otherpath',
  'This is a great other summary!')]

In [46]:
podcast_db.query_db("SELECT pid, url, podcast_title from podcast_details WHERE pid = 1")

(1, 'https://newurl.com', 'some_podcast_title')


[(1, 'https://newurl.com', 'some_podcast_title')]

In [16]:
update_dict = {"url" : "https://newurl.com", "podcast_name" : "nicer_name", "summary" : "A much better summary today!"}
podcast_db.update_db(1, update_dict)

UPDATE podcast_details SET url = 'https://newurl.com', podcast_name = 'nicer_name', summary = 'A much better summary today!' WHERE pid = 1
Record with pid = 1 has been successfully updated.


In [17]:
podcast_db.query_db("all")

(1, 'https://newurl.com', 'some_podcast_title', 'nicer_name', 'dir/transcript/path', 'dir/embedding/path', 'dir/summary_path', 'A much better summary today!')
(2, 'https://other_podcast.com', 'other_podcast_title', 'other_podcast_name', 'dir/transcript/otherpath', 'dir/embedding/otherpath', 'dir/summary_otherpath', 'This is a great other summary!')


[(1,
  'https://newurl.com',
  'some_podcast_title',
  'nicer_name',
  'dir/transcript/path',
  'dir/embedding/path',
  'dir/summary_path',
  'A much better summary today!'),
 (2,
  'https://other_podcast.com',
  'other_podcast_title',
  'other_podcast_name',
  'dir/transcript/otherpath',
  'dir/embedding/otherpath',
  'dir/summary_otherpath',
  'This is a great other summary!')]

#### How does the table 1 and table 2 update work. Do we have to manualy insert in both or just table 2.

In [40]:
podcast_1 = (12333,)
podcast_2 = (98,)
podcast_views = [podcast_1, podcast_2]

In [41]:
podcast_views

[(12333,), (98,)]

In [42]:
podcast_db.insert_podcast(podcast_views, which_table = "first")

podcast_views
2 records successfully inserted.


In [43]:
podcast_db.query_db("SELECT * from podcast_views")

(1, 12333)
(2, 98)


[(1, 12333), (2, 98)]